In [1]:
# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader

In [2]:
import json
import urllib

url = "https://raw.githubusercontent.com/Chaoyuuu/Gather-Town-Datasets/master/datasets-beautiful-rooms.json"
response = urllib.request.urlopen(url)
data = json.loads(response.read())
data_length = len(data)
input_data = torch.zeros((data_length, 8, 13, 10))

"""
for i in range(data_length):
    room_id = data[i]["id"]
    if "Ning" not in room_id and "Han" not in room_id: 
        count += 1
        print(room_id)
        
print("count: " + str(data_length))
"""
    

# room_decoration
encode_size = 11
room_decoration_data = torch.zeros((data_length, encode_size, 13, 10)) # (7 object + 2 orientation)
room_decoration_label_dict = {
    "Whiteboard": 0,
    "Projector Screen": 1,
    "TV (Flatscreen)": 2,
    "Bookshelf (2x4)": 3,
    "Bookshelf [Tall] (1x2)": 4,
    "Potted Plant (Spikey)": 5,
    "Lucky Bamboo": 6,
    "Mod Chair": 7,
    "Captain's Chair": 8,
}

for d in range(data_length):
    i = d
    room = data[d]["room"]
    for k in range(len(room)):
        object = room[k]
        name = object["_name"]
        if name in room_decoration_label_dict:
            label = room_decoration_label_dict[name]
            x = object["x"]
            y = object["y"]
#             print(f"({x}, {y})")
            if x >= 10:
                x = x -1
            orientation = object["orientation"]

            room_decoration_data[i][label][y][x] = 1
#             print("exist: " + name)
#             print(table_arrange_data[i][label])

            if orientation == 0:
                room_decoration_data[i][encode_size - 1][y][x] = -1
            elif orientation == 1:
                room_decoration_data[i][encode_size - 2][y][x] = -1
            elif orientation == 2:
                room_decoration_data[i][encode_size - 1][y][x] = 1
            else:
                room_decoration_data[i][encode_size - 2][y][x] = 1
                    

z_dim = 8
x_dim = encode_size*13*10
bs = 60
epoch_round = 10
print("datasets number = " + str(data_length))


train_loader = DataLoader(dataset=room_decoration_data, batch_size=bs, shuffle=True)

datasets number = 246


In [3]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        
        # encoder part
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc31 = nn.Linear(h_dim2, z_dim)
        self.fc32 = nn.Linear(h_dim2, z_dim)
        # decoder part
        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
        
    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h) # mu, log_var
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return torch.sigmoid(self.fc6(h)) 
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, x_dim))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

# build model
vae = VAE(x_dim=x_dim, h_dim1= 512, h_dim2=128, z_dim=z_dim)
if torch.cuda.is_available():
    vae.cuda()

In [4]:
vae

VAE(
  (fc1): Linear(in_features=1430, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc31): Linear(in_features=128, out_features=8, bias=True)
  (fc32): Linear(in_features=128, out_features=8, bias=True)
  (fc4): Linear(in_features=8, out_features=128, bias=True)
  (fc5): Linear(in_features=128, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=1430, bias=True)
)

In [5]:
optimizer = optim.Adam(vae.parameters())
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, x_dim), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

In [6]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, data in enumerate(train_loader):
        data = data.cuda()
        optimizer.zero_grad()
        
        print(list(data.size())) # [81, 17, 13, 10]
        
        recon_batch, mu, log_var = vae(data) #torch.float32
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [7]:
def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.cuda()
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [8]:
for epoch in range(1, epoch_round):
    train(epoch)
#     test()

[60, 11, 13, 10]
Train Epoch: 1 [0/246 (0%)]	Loss: 992.021745
[60, 11, 13, 10]
[60, 11, 13, 10]
[60, 11, 13, 10]
[6, 11, 13, 10]
====> Epoch: 1 Average loss: 922.2383
[60, 11, 13, 10]
Train Epoch: 2 [0/246 (0%)]	Loss: 732.619727
[60, 11, 13, 10]
[60, 11, 13, 10]
[60, 11, 13, 10]
[6, 11, 13, 10]
====> Epoch: 2 Average loss: 611.9237
[60, 11, 13, 10]
Train Epoch: 3 [0/246 (0%)]	Loss: 318.995247
[60, 11, 13, 10]
[60, 11, 13, 10]
[60, 11, 13, 10]
[6, 11, 13, 10]
====> Epoch: 3 Average loss: 226.4118
[60, 11, 13, 10]
Train Epoch: 4 [0/246 (0%)]	Loss: 41.771057
[60, 11, 13, 10]
[60, 11, 13, 10]
[60, 11, 13, 10]
[6, 11, 13, 10]
====> Epoch: 4 Average loss: 19.1301
[60, 11, 13, 10]
Train Epoch: 5 [0/246 (0%)]	Loss: -11.888725
[60, 11, 13, 10]
[60, 11, 13, 10]
[60, 11, 13, 10]
[6, 11, 13, 10]
====> Epoch: 5 Average loss: -20.7580
[60, 11, 13, 10]
Train Epoch: 6 [0/246 (0%)]	Loss: -48.415975
[60, 11, 13, 10]
[60, 11, 13, 10]
[60, 11, 13, 10]
[6, 11, 13, 10]
====> Epoch: 6 Average loss: -82.9390


In [42]:
with torch.no_grad():
    z = torch.randn(1, z_dim).cuda()
    sample = vae.decoder(z).cuda()
    
    torch.set_printoptions(profile="full")    
    data = sample[0]
    
    output_tensor = torch.zeros((encode_size, 13, 10))
    threshold = 0.27
    
    # reshape sample(1*200) -> output_tensor(17, 13, 10)
    for i, batch in enumerate(sample):
        i = 0
        for depth in range(encode_size):
            for col in range(13):
                for row in range(10):
                    #print("depth, col, row: " + str(row)+", " + str(col) +", ", str(depth))
                    data = batch[i]
                    i += 1
                    if data >= threshold:
                        output_tensor[depth][col][row] = data
#                     if depth == 8 or depth == 7 or depth == 4:
#                         if data >= threshold - 0.05:
#                             output_tensor[depth][col][row] = data
    
    object_dict = {
        0: "WB",
        1: "PR",
        2: "TV",
        3: "B4",
        4: "B2",
        5: "PP",
        6: "LB",
        7: "MC",
        8: "CC"
    }


    room = [['--']*10 for i in range(13)]
    for col in range(13):
        for row in range(10):
            object_duplicate_flage = 0
            max_value = 0
            max_index = -1
            for depth in range(encode_size - 2):
                data = output_tensor[depth][col][row]
                
                if max_value < data:
                    max_index = depth
                                        
            if max_index != -1:
                room[col][row] = object_dict[max_index]
    
    for col in range(13):
        print(room[col])
        
        
#     print(sample)
                    
    # print room with object
    # WB: Whiteboard
    # PR: Projector Screen
    # T2: Chippendale Table (2x3)
    # TV: TV (Flatscreen)
    # B4: Bookshelf (2x4)
    # PP: Potted Plant (Spikey)
    # MC: Mod Chair
    # CC: Captain’s Chair
    # CS: Chair (Simple)
    # T3: Chippendale Table (3x3)
    # B2: Bookshelf [Tall] (1x2)
    # LP: Laptop
    # MP: Microphone
    # LB: Lucky Bamboo
    # DC: Dining Chair (Square)

['PP', '--', 'B4', 'TV', 'WB', '--', '--', '--', '--', 'PP']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['B4', '--', '--', '--', 'MC', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['PP', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['PP', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', 'PP']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['PP', '--', '--', '--', '--', '--', '--', '--', '--', 'PP']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']
['--', '--', '--', '--', '--', '--', '--', '--', '--', '--']


In [40]:
# PATH = "./outputModel/vae-room"

# # Save
# torch.save({
#     'model': vae,
#     'z_dim': z_dim,
#     'threshold': threshold,
#     'encode_size': encode_size
#     }, PATH)